# 交通事故のエクセルデータをＤＢへ登録する

## 交通事故のオープンデータ

 - https://ckan.open-governmentdata.org/dataset?q=%E4%BA%A4%E9%80%9A%E4%BA%8B%E6%95%85
 
 - "福岡県　平成●●年　交通事故.xlsx"をダウンロード
 
 - 以下のコードをエクセルのブック名を変更した後に、実行

In [ ]:
import sqlite3
import pandas as pd

print('Start')

# 読み込むエクセル
EXCEL = '●●.xlsx'

# データベースファイルのパス
dbpath = 'accident.sqlite'
 
# データベースのレコード数を取得し、ｉｄの初期値とする
conn = sqlite3.connect(dbpath)
query = "select count(*) from master;"
id_start = pd.read_sql_query(query, conn).iloc[0][0]

print('Read Excel')
# エクセルのデータからデータフレームｄｆを生成（ｎａｎは、０とする）
df = pd.read_excel(EXCEL)
df.fillna(0, inplace=True)

# 読み込んだデータフレームｄｆのｉｄを付け替える
idx = id_start
for i, data in df.iterrows():
    df.iloc[[i],[0]] = idx + i

print('Write DB')
# ｄｆのデータをデータベースに格納
# ＤＢ格納では、Pandasは遅いので、sqlite3を使用
connection = sqlite3.connect(dbpath)
cursor = connection.cursor()
 
try:
    for i, data in df.iterrows():
        cts = []
        for d in data:
            cts.append(d)
        text = "INSERT INTO master VALUES " + str(tuple(cts)) 
        cursor.execute(text)
except sqlite3.Error as e:
    print('sqlite3.Error occurred:', e.args[0])

connection.commit()

print('End')

In [ ]:
conn = sqlite3.connect('accident.sqlite')
query = "select * from master;"
df = pd.read_sql_query(query, conn)

df.head()

In [ ]:
df.describe()

In [ ]:
df[df['injury']==27].T